In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Data loading

import os
for dirname, _, filenames in os.walk('/kaggle/input/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/fake-news/submit.csv
/kaggle/input/fake-news/train.csv
/kaggle/input/fake-news/test.csv


In [3]:
df = pd.read_csv(r"/kaggle/input/fake-news/train.csv")

In [4]:
df
# 0 means not fake and 1 means fake

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [5]:
percentage_null_values = (df.isnull().sum()/df.isnull().count())*100
percentage_null_values

id        0.000000
title     2.682692
author    9.408654
text      0.187500
label     0.000000
dtype: float64

In [6]:
# as we have to work only with title so its better to remove null values
df = df.dropna()

In [7]:
df.isnull().sum()
# null values removed

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [8]:
x = df.drop("label" , axis = 1)

In [9]:
y = df['label']

In [10]:
print("x_shape:",x.shape)
print("y_shape:",y.shape)

x_shape: (18285, 4)
y_shape: (18285,)


In [11]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [12]:
vocab_size = 5000          # dictionary of words

### Step:
### text data--->>preprocessing(removing stop words etc)--->>onhot(to get index)--->>embedding layer(to get setiment vectors)    

## Preprocessing data

In [13]:
data = x.copy()
data.head(2)

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...


In [14]:
data.reset_index(inplace = True)   # as we removed null values so have to reset index 

In [15]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [16]:
stop = stopwords.words('english')

In [17]:
ps = PorterStemmer()
corpus = []

for i in range(len(data)):
    final = re.sub('[^a-zA-Z]',' ',data['title'][i])
    final = final.lower()
    final = final.split()
    final = [ps.stem(word) for word in final if not word in stop]
    final = ' '.join(final)
    corpus.append(final)

In [18]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

## one-hot

In [19]:
voc_size = 5000

one_hot = [one_hot(words ,voc_size) for words in corpus] # to get index of all words

In [20]:
one_hot

[[2312, 2028, 4430, 3083, 2684, 4789, 225, 1133, 4625, 280],
 [2435, 3506, 1690, 2693, 4002, 962, 752],
 [1911, 4484, 4481, 4349],
 [1006, 703, 99, 580, 135, 3115],
 [4437, 4002, 3157, 4485, 2467, 2908, 4002, 1032, 1391, 3829],
 [3827,
  2104,
  1614,
  1853,
  1988,
  615,
  989,
  3160,
  1143,
  1295,
  2198,
  715,
  2744,
  4831,
  752],
 [3727, 4906, 4395, 4693, 471, 618, 3506, 1945, 1610, 3990, 1101],
 [534, 272, 1866, 754, 2918, 1869, 615, 3229, 1610, 3990, 1101],
 [2484, 1739, 3202, 2061, 2044, 856, 4404, 3745, 615, 127],
 [3929, 4503, 3181, 2195, 2983, 1859, 4611, 3867],
 [2640, 93, 3707, 324, 4914, 4869, 729, 403, 1118, 793, 3080],
 [580, 1008, 2684, 856, 615, 2918],
 [2523, 3908, 3297, 897, 281, 205, 956, 3307, 4472],
 [1664, 4525, 4973, 2327, 4912, 755, 252, 1610, 3990, 1101],
 [4723, 2906, 1900, 2544, 1780, 1610, 3990, 1101],
 [3003, 3944, 2020, 2435, 3870, 2575, 1201, 4901, 4904, 4444],
 [910, 1306, 3506],
 [277, 50, 4067, 1100, 615, 3477, 1854, 752],
 [4634, 952, 1690, 

## Embedding layer

In [21]:
# we will do padding to make all lengths same 

embedded_rep = pad_sequences(one_hot , padding = 'pre' , maxlen = 20)
embedded_rep

array([[   0,    0,    0, ..., 1133, 4625,  280],
       [   0,    0,    0, ..., 4002,  962,  752],
       [   0,    0,    0, ..., 4484, 4481, 4349],
       ...,
       [   0,    0,    0, ..., 1610, 3990, 1101],
       [   0,    0,    0, ...,  604,   11, 4464],
       [   0,    0,    0, ...,  995, 2468, 1396]], dtype=int32)

In [22]:
embedded_rep[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 2312,
       2028, 4430, 3083, 2684, 4789,  225, 1133, 4625,  280], dtype=int32)

In [23]:
embedded_rep.shape

(18285, 20)

In [24]:
# making model
from keras.layers import Dropout

embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size , embedding_vector_features , input_length = 20))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1 , activation = 'sigmoid'))
model.compile(optimizer = 'adam' , loss = 'binary_crossentropy' , metrics = ['accuracy'])

In [25]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
dropout (Dropout)            (None, 20, 40)            0         
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________


In [26]:
# now we will train model and for that we will make our x and y

x_final = np.array(embedded_rep)
y_final = np.array(y)

In [27]:
# splitting data for train and test

from sklearn.model_selection import train_test_split

x_train , x_test , y_train , y_test = train_test_split(x_final , y_final , random_state = 0 , test_size = 0.30)

print("shape of x_train:",x_train.shape)
print("shape of x_test:",x_test.shape)
print("shape of y_train:",y_train.shape)
print("shape of y_test:",y_test.shape)

shape of x_train: (12799, 20)
shape of x_test: (5486, 20)
shape of y_train: (12799,)
shape of y_test: (5486,)


In [28]:
# model training

model.fit(x = x_train , y = y_train , validation_data = (x_test , y_test) , epochs = 10 , batch_size = 64)

Epoch 1/10
200/200 [==============================] - 9s 36ms/step - loss: 0.4854 - accuracy: 0.7595 - val_loss: 0.1938 - val_accuracy: 0.9160
Epoch 2/10
200/200 [==============================] - 7s 33ms/step - loss: 0.1522 - accuracy: 0.9427 - val_loss: 0.1916 - val_accuracy: 0.9171
Epoch 3/10
200/200 [==============================] - 7s 33ms/step - loss: 0.1053 - accuracy: 0.9622 - val_loss: 0.2000 - val_accuracy: 0.9180
Epoch 4/10
200/200 [==============================] - 7s 33ms/step - loss: 0.0788 - accuracy: 0.9748 - val_loss: 0.2219 - val_accuracy: 0.9183
Epoch 5/10
200/200 [==============================] - 7s 33ms/step - loss: 0.0580 - accuracy: 0.9811 - val_loss: 0.2534 - val_accuracy: 0.9143
Epoch 6/10
200/200 [==============================] - 6s 32ms/step - loss: 0.0449 - accuracy: 0.9856 - val_loss: 0.3402 - val_accuracy: 0.8972
Epoch 7/10
200/200 [==============================] - 7s 34ms/step - loss: 0.0346 - accuracy: 0.9890 - val_loss: 0.2816 - val_accuracy: 0.9172

In [29]:
y_pred = model.predict_classes(x_test)

/opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [30]:
y_pred

array([[1],
       [1],
       [1],
       ...,
       [0],
       [1],
       [0]], dtype=int32)

In [31]:
y_test

array([0, 0, 1, ..., 0, 1, 0])

In [32]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test,y_pred)

0.9101348888078746

In [33]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test,y_pred)

array([[2812,  272],
       [ 221, 2181]])